# Baffin Bay Pb concentration along cross section

In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import cmocean
from math import sin, cos, sqrt, atan2, radians
import xarray as xr
import imageio
from datetime import datetime
import glob

%matplotlib inline

#### Parameters

In [2]:
# domain dimensions:
imin, imax = 1139, 2179
jmin, jmax = 159, 799

# transect coordinates:
BB_coords = (1470,1471,515,670) # Baffin Bay

In [3]:
# Colors
land_color = '#a9a7a2'
light_gray = '#b1b1b1'
light_land = '#d0d0cf'
land_edge  = '#929291'

#### Files

In [4]:
mesh         = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mask_Pb-20230213.nc')
mesh_lon     = np.array(mesh.variables['nav_lon'])
mesh_lat     = np.array(mesh.variables['nav_lat'])
mesh_bathy   = np.array(mesh.variables['tmask'][0,:,:,:])
bathy_masked = np.ma.masked_where((mesh_bathy> 0.1), mesh_bathy)
ocean_masked = np.ma.masked_where((mesh_bathy<0.1), mesh_bathy)
depths       = np.array(mesh.variables['nav_lev'])

In [5]:
# transect coordinates
BB_lons = mesh_lon[BB_coords[0]:BB_coords[1],BB_coords[2]:BB_coords[3]][0,:]
BB_lats = mesh_lat[BB_coords[0]:BB_coords[1],BB_coords[2]:BB_coords[3]][0,:]
BB_deps = bathy_masked[:,BB_coords[0]:BB_coords[1],BB_coords[2]:BB_coords[3]][:,0,:]

In [6]:
years  = np.arange(2004,2020,1) 
files1 = []; files2 = [];
for year in years:
    folder1 = f'/data/brogalla/run_storage/Pb-reference-202303/Pb-reference-{year}_20230308/'
    folder2 = f'/data/brogalla/run_storage/Pb-reference-202303/Pb-Labrador-Sea-{year}_20230308/'
        
    files1 = np.append(files1, np.sort(glob.glob(f'{folder1}/ANHA12_EXH006_5d_*')))
    files2 = np.append(files2, np.sort(glob.glob(f'{folder2}/ANHA12_EXH006_5d_*')))
    
dates1 = [file.split('ptrc_T_')[1][0:-3] for file in files1]
dates2 = [file.split('ptrc_T_')[1][0:-3] for file in files2]

#### Functions

In [7]:
# Convert lat/lon to distance along transect:
def calc_distance(lats, lons):
    R = 6373.0 # approximate radius of earth in km

    lats_rad = [radians(l) for l in lats]
    lons_rad = [radians(l) for l in lons]

    dlon = [lon_step - lons_rad[0] for lon_step in lons_rad]
    dlat = [lat_step - lats_rad[0] for lat_step in lats_rad]

    distance = np.empty((len(dlon)))
    for i in range(0,len(dlon)):
        a = sin(dlat[i] / 2)**2 + cos(lats_rad[0]) * cos(lats_rad[i]) * sin(dlon[i] / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance[i] = R * c
        
    return distance

In [8]:
def plot_cross_section(file1, n, vmin=0, vmax=30):
    
    with xr.open_dataset(f'{file1}') as f:
        f1 = f['dissolpb'].values[0,:,BB_coords[0]-imin:BB_coords[1]-imin,BB_coords[2]-jmin:BB_coords[3]-jmin][:,0,:]
        
    fig, ax = plt.subplots(1,1, figsize=(4.5, 3.7*0.8), dpi=300)

    ax.set_xlabel('Distance [km]', fontsize=7)
    ax.set_ylabel('Depth [m]', fontsize=7)
    ax.invert_yaxis()
    ax.tick_params(axis='both', which='major', labelsize=6)
    ax.set_ylim(2600,-10)

    # Cross-section
    xx, zz = np.meshgrid(XS_distance, depths)
    CB1 = ax.pcolormesh(xx, zz, f1*1e12, vmin=vmin, vmax=vmax, cmap=cmocean.cm.matter)
    ax.pcolormesh(xx, zz, BB_deps, cmap=matplotlib.colors.ListedColormap([land_color]))

    cbax1 = fig.add_axes([0.75, 0.2, 0.04, 0.3]);
    CBar  = plt.colorbar(CB1, ax=ax, cax=cbax1, ticks=[0,5,10,15,20]); 
    CBar.set_label('Dissolved Pb [pM]', fontsize=7)
    CBar.ax.tick_params(axis='y', length=0, labelsize=6)
   
    ax.set_title(datetime.strptime(dates1[n][0:8],"%Y%M%d").strftime('%Y-%M-%d'), fontsize=6)
    return fig

In [9]:
def plot_cross_section_Atlantic(file1, file2, n, vmin=0, vmax=30):
    
    with xr.open_dataset(f'{file1}') as f:
        f1 = f['dissolpb'].values[0,:,BB_coords[0]-imin:BB_coords[1]-imin,BB_coords[2]-jmin:BB_coords[3]-jmin][:,0,:]
    with xr.open_dataset(f'{file2}') as f:
        f2 = f['dissolpb'].values[0,:,BB_coords[0]-imin:BB_coords[1]-imin,BB_coords[2]-jmin:BB_coords[3]-jmin][:,0,:]
    
    fig, ax = plt.subplots(1,1, figsize=(4.5, 3.7*0.8), dpi=300)

    ax.set_xlabel('Distance [km]', fontsize=7)
    ax.set_ylabel('Depth [m]', fontsize=7)
    ax.invert_yaxis()
    ax.tick_params(axis='both', which='major', labelsize=6)
    ax.set_ylim(2600,-10)

    # Cross-section
    xx, zz = np.meshgrid(XS_distance, depths)
    CB1 = ax.pcolormesh(xx, zz, (f2-f1)*1e12, vmin=vmin, vmax=vmax, cmap=cmocean.cm.rain)
    ax.pcolormesh(xx, zz, BB_deps, cmap=matplotlib.colors.ListedColormap([land_color]))

    cbax1 = fig.add_axes([0.75, 0.2, 0.04, 0.3]);
    CBar  = plt.colorbar(CB1, ax=ax, cax=cbax1, ticks=[0,0.5,1,1.5]); 
    CBar.set_label(r'$\Delta$ Dissolved Pb [pM]', fontsize=7)
    CBar.ax.tick_params(axis='y', length=0, labelsize=6)

    ax.set_title(datetime.strptime(dates1[n][0:8],"%Y%M%d").strftime('%Y-%M-%d'), fontsize=6)
    
    return fig

#### Calculations

In [10]:
XS_distance  = calc_distance(BB_lats, BB_lons)   

#### Figures

In [ ]:
for n, date in enumerate(dates1):
    fig = plot_cross_section_Atlantic(files1[n], files2[n], n, vmin=0, vmax=1.5)
    fig.savefig(f'/ocean/brogalla/GEOTRACES/figures/Pb/animation/Baffin-Bay-cross-section-Atlantic-{date[0:8]}.png', 
                dpi=300)
    plt.close(fig)

In [ ]:
filenames = np.sort(glob.glob(f'/ocean/brogalla/GEOTRACES/figures/Pb/animation/Baffin-Bay-cross-section-Atlantic*'))
with imageio.get_writer(f'/ocean/brogalla/GEOTRACES/figures/Pb/animation/Baffin-Bay-cross-section-Atlantic.mp4', 
                        macro_block_size = None, fps=7, mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

In [74]:
for n, date in enumerate(dates1):
    fig = plot_cross_section(files1[n], n, vmin=0, vmax=20)
    fig.savefig(f'/ocean/brogalla/GEOTRACES/figures/Pb/animation/Baffin-Bay-cross-section-{date[0:8]}.png', dpi=300)
    plt.close(fig)

In [75]:
filenames = np.sort(glob.glob(f'/ocean/brogalla/GEOTRACES/figures/Pb/animation/Baffin-Bay-cross-section-*'))
with imageio.get_writer(f'/ocean/brogalla/GEOTRACES/figures/Pb/animation/Baffin-Bay-cross-section.mp4', 
                        macro_block_size = None, fps=7, mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)